## 문자 레벨 기계 번역기 구현하기

-> 병렬 코퍼스 필요 : 두 개 이상의 언어가 병렬적으로 구성된 코퍼스

In [1]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [3]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))


전체 샘플의 개수 : 26354


In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)


,src,tar
11759,I'd like a hug.,J'aimerais un câlin.
9684,Tom will work.,Tom travaillera.
13983,You seem upset.,Vous semblez contrariée.
11012,Hello everyone!,Salut les gens !
25466,Do as you see fit.,Faites comme bon vous semble.
20972,I pay my own way.,Je peux me gérer seul.
5791,Is that snow?,Est-ce que c'est de la neige ?
14837,Go to the store.,Va au magasin !
25195,Are you up for it?,Y es-tu prêt ?
17838,Tom is perverse.,Tom est pervers.


In [5]:
lines.tar=lines.tar.apply(lambda x : '\t' + x + '\n')
lines.sample(10)

,src,tar
9500,Tom is family.,\tTom fait partie de la famille.\n
16195,I'm often wrong.,\tJe me trompe souvent.\n
9220,That's ironic.,\tC'est ironique.\n
8867,Lock the safe.,\tVerrouillez le coffre-fort.\n
8195,I woke you up.,\tJe t'ai réveillé.\n
8021,I outrank you.,\tJe passe avant vous.\n
7551,He's no saint.,\tCe n'est pas un saint.\n
7475,He is awesome.,\tIl est génial.\n
5457,I won't come.,\tJe ne viendrai pas.\n
18842,You're adorable.,\tVous êtes adorables.\n


In [6]:
#문자 집합 구축
src_vocab=set()
for line in lines.src:
  for char in line:
    src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

In [7]:
src_vocab_size=len(src_vocab)+1
tar_vocab_size=len(tar_vocab)+1
print('source 문장의 char 집합 :', src_vocab_size)
print('target 문장의 char 집합 :', tar_vocab_size)

source 문장의 char 집합 : 77
target 문장의 char 집합 : 104


In [8]:
src_vocab=sorted(list(src_vocab))
tar_vocab=sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x']


In [9]:
#각 문자에 인덱스 부여
src_to_index=dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index=dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 3

In [10]:
#정수 인코딩
#영어먼저

encoder_input=[]

#1개의 문장
for line in lines.src:
  encoded_line=[]
  for char in line:
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩:',encoder_input[:5])

source 문장의 정수 인코딩: [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [11]:
#프랑스어 정수 인코딩
decoder_input=[]
for line in lines.tar:
  encoded_line=[]
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩:',decoder_input[:5])

target 문장의 정수 인코딩: [[1, 48, 52, 3, 4, 2], [1, 39, 52, 69, 54, 59, 56, 14, 2], [1, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 2], [1, 28, 66, 72, 58, 56, 3, 4, 2], [1, 45, 52, 63, 72, 71, 3, 4, 2]]


In [12]:
#정수 인코딩 과정에서 <sos> => 프랑스어 맨 앞에 붙어있는 " 제거 가좡
decoder_target=[]
for line in lines.tar:
  timestep =0
  encoded_line=[]
  for char in line:
    if timestep>0:
      encoded_line.append(tar_to_index[char])
    timestep=timestep+1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩:',decoder_target[:5])

target 문장 레이블의 정수 인코딩: [[48, 52, 3, 4, 2], [39, 52, 69, 54, 59, 56, 14, 2], [31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 2], [28, 66, 72, 58, 56, 3, 4, 2], [45, 52, 63, 72, 71, 3, 4, 2]]


In [13]:
#패딩 작업 진행
#영어랑 프랑스어 따로따로
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이: ',max_src_len)
print('target 문장의 최대 길이: ',max_tar_len)

source 문장의 최대 길이:  18
target 문장의 최대 길이:  59


In [14]:
#padding 작업
encoder_input=pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input=pad_sequences(decoder_input,maxlen=max_tar_len, padding='post')
decoder_target=pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [15]:
#원 핫 인코딩 수행
encoder_input=to_categorical(encoder_input)
decoder_input=to_categorical(decoder_input)
decoder_target=to_categorical(decoder_target)

In [16]:
##seq2seq 모델 설계
#훈련시키기
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [17]:
encoder_inputs=Input(shape=(None, src_vocab_size))
encoder_lstm=LSTM(units=256, return_state=True)

#encoder_outputs는 여기서 불필요함
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

#LSTM은 바닐라 RNN과는 달리 상태가 두개, 은닉상태와 셀 상태
encoder_states=[state_h,state_c]

In [18]:
decoder_inputs=Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

#디코더에게 인코더의 은닉 상태, 셀 상태를 전달
decoder_outputs,_,_ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer =Dense(tar_vocab_size, activation='softmax')
decoder_outputs=decoder_softmax_layer(decoder_outputs)

model=Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [19]:
model.fit(x=[encoder_input, decoder_input],y=decoder_target,batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
330/330 [==============================] - 16s 19ms/step - loss: 1.1524 - val_loss: 1.0427
Epoch 2/40
330/330 [==============================] - 4s 13ms/step - loss: 0.7771 - val_loss: 0.8489
Epoch 3/40
330/330 [==============================] - 7s 22ms/step - loss: 0.6721 - val_loss: 0.7744
Epoch 4/40
330/330 [==============================] - 8s 23ms/step - loss: 0.6166 - val_loss: 0.7225
Epoch 5/40
330/330 [==============================] - 6s 19ms/step - loss: 0.5765 - val_loss: 0.6743
Epoch 6/40
330/330 [==============================] - 5s 15ms/step - loss: 0.5422 - val_loss: 0.6500
Epoch 7/40
330/330 [==============================] - 5s 17ms/step - loss: 0.5144 - val_loss: 0.6220
Epoch 8/40
330/330 [==============================] - 6s 20ms/step - loss: 0.4913 - val_loss: 0.5953
Epoch 9/40
330/330 [==============================] - 6s 17ms/step - loss: 0.4729 - val_loss: 0.5836
Epoch 10/40
330/330 [==============================] - 5s 15ms/step - loss: 0.4562 - val_l

In [20]:
#seq2seq 기계 번역기 동장시키기
encoder_model =Model(inputs=encoder_inputs, outputs=encoder_states)

In [21]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)


In [22]:
#인덱스로부터 단어를 얻을 수 있는 index_to_src와 index_to_tar
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [23]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence


In [24]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력


1/1 [==============================] - 0s 22ms/step
-----------------------------------
입력 문장: Go.
정답 문장: ouge !
번역 문장: ars !
1/1 [==============================] - 0s 25ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: onjour !
번역 문장: idez-le.
1/1 [==============================] - 0s 23ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: 'ai pigé !
번역 문장: arde-le.
1/1 [==============================] - 0s 51ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: entre à la maison.
번역 문장: a au travail !
1/1 [==============================] - 0s 20ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: n avant.
번역 문장: écampe !


## Word-Level 번역기 만들기


In [25]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


In [26]:
num_samples = 33000


In [27]:
#구두점 등을 제거하거나 단어와 구분해주기 위한 전처리

def to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
  # 악센트 제거 함수 호출
  sent = to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 추가.
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent


In [28]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))


전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [29]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target


In [30]:
 sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])


인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


In [31]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")


In [32]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)


인코더의 입력의 크기(shape) : (33000, 7)
디코더의 입력의 크기(shape) : (33000, 16)
디코더의 레이블의 크기(shape) : (33000, 16)


In [33]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))


영어 단어 집합의 크기 : 4481, 프랑스어 단어 집합의 크기 : 7873


In [34]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)


랜덤 시퀀스 : [23316 24063 22360 ... 22458  6958  2022]


In [35]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]


In [36]:
encoder_input[30997]


array([825,   1,   0,   0,   0,   0,   0], dtype=int32)

In [37]:
decoder_input[30997]


array([   2, 4729,    9,    1,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [38]:
decoder_target[30997]


array([4729,    9,    1,    3,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [39]:
## 이때 decoder_input과 decoder_target의 값은 같아야함 !! => 교사 강요 **

In [40]:
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)


검증 데이터의 개수 : 3300


In [41]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]


In [42]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)


훈련 source 데이터의 크기 : (29700, 7)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 7)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


### 모델 설계
- Masking은 패딩 토큰인 숫자 0의 경우에는 연산을 제외하는 역할
- 인코더의 내부 상태를 디코더로 넘겨주어야 하기 때문에 return_state=True
- 인코더에 입력을 넣으면 내부 상태를 리턴
- LSTM에서 state_h, state_c를 리턴받는데, 이는 각각 RNN 챕터에서 LSTM을 처음 설명할 때 언급하였던 은닉 상태와 셀 상태에 해당. 이 두 가지 상태를 encoder_states에 저장, encoder_states를 디코더에 전달하므로서 이 두 가지 상태 모두를 디코더로 전달 => 컨텍스트 벡터

In [43]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [44]:
embedding_dim=64
hidden_units=64

In [45]:
#인코더 설계
encoder_inputs=Input(shape=(None,))
#임베딩 층
enc_emb=Embedding(src_vocab_size,embedding_dim)(encoder_inputs)
#패딩 0은 연산에서 제외
enc_masking=Masking(mask_value=0.0)(enc_emb)
#상태값 리턴을 위해 return_state 는 True
encoder_lstm=LSTM(hidden_units, return_state=True)
#은닉 상태와 셀 상태를 리턴
encoder_outputs, state_h, state_c=encoder_lstm(enc_masking)
#인코더의 은닉 상태와 셀 상태를 저장
encoder_states=[state_h,state_c]

In [46]:
#디코더 설계
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) #임베딩 층
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking=Masking(mask_value=0.0)(dec_emb) #패딩0은 연산에서 제외

#상태값 리턴을 위해 return_state= True, 모든 시점에 대해 단어를 예측하기 위해 return_sequences=True
decoder_lstm=LSTM(hidden_units, return_sequences=True,return_state=True)

#인코더의 은닉 상태를 초기 은닉 상태 (initial_state)로 사용
decoder_outputs,_,_=decoder_lstm(dec_masking,initial_state=encoder_states)

#모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense=Dense(tar_vocab_size, activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

#모델의 입력과 출력을 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [47]:
#모델 훈련
#batch_size = 128
#epoch = 50

model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train,\
          validation_data=([encoder_input_test, decoder_input_test],
                           decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
233/233 [==============================] - 41s 131ms/step - loss: 3.3225 - acc: 0.6137 - val_loss: 1.9527 - val_acc: 0.6262
Epoch 2/50
233/233 [==============================] - 10s 42ms/step - loss: 1.8191 - acc: 0.6748 - val_loss: 1.7148 - val_acc: 0.7226
Epoch 3/50
233/233 [==============================] - 8s 35ms/step - loss: 1.6445 - acc: 0.7425 - val_loss: 1.5740 - val_acc: 0.7545
Epoch 4/50
233/233 [==============================] - 10s 41ms/step - loss: 1.5244 - acc: 0.7560 - val_loss: 1.4723 - val_acc: 0.7658
Epoch 5/50
233/233 [==============================] - 9s 39ms/step - loss: 1.4241 - acc: 0.7652 - val_loss: 1.3918 - val_acc: 0.7735
Epoch 6/50
233/233 [==============================] - 8s 35ms/step - loss: 1.3401 - acc: 0.7801 - val_loss: 1.3163 - val_acc: 0.7879
Epoch 7/50
233/233 [==============================] - 8s 34ms/step - loss: 1.2658 - acc: 0.7910 - val_loss: 1.2589 - val_acc: 0.7941
Epoch 8/50
233/233 [==============================] - 8s 35ms/ste

## 기계 번역기 동작시키기

In [48]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [49]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence


In [54]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [52]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)


1/1 [==============================] - 0s 35ms/step


KeyError: ignored

In [ ]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)
